# PTT Hate版資料爬蟲

## Install Required Packages

In [7]:
from __future__ import absolute_import
from __future__ import print_function

import os
import re
import sys
import json
import csv
import time
from datetime import datetime
import pandas as pd

import requests
from bs4 import BeautifulSoup
from six import u

全爬下來的話：

1. 文章有「公告」，做停權處置等等，暫不處理
2. 截至2/16共有3990個分頁
3. 目前只爬文章內容，不爬推噓文

## ptt-Crawler 

In [ ]:
PTT_URL = 'https://www.ptt.cc'
board = 'Hate'
start_index = 1
end_index = 3990
all_articles = []

for i in range(end_index - start_index + 1):
    index = start_index + i
    print('Processcing index: ', str(index))
    
    resp = requests.get(url = PTT_URL + '/bbs/' + board + '/index' + str(index) + '.html',
                        cookies = {'over18':'1'})
    
    # status_code=200 is success, so if not success will return error e.g. 404 not found
    if resp.status_code != 200:
        print('invalid url: ', resp.url)
        continue
    
    # parser the html and find articles' div element
    soup = BeautifulSoup(resp.text, 'html.parser')
    divs = soup.find_all('div', 'r-ent')
    
    for div in divs:
        try:
            # for link
            href = div.find('a')['href']
            link = PTT_URL + href
            article_id = re.sub('\.html', '', href.split('/')[-1])

            # print('Processing article: ', article_id)
            # print('link', link) this is for test 
            resp2 = requests.get(url = link, cookies={'over18':'1'})
            
            # status_code=200 is success, so if not success will return error e.g. 404 not found
            if resp2.status_code != 200:
                print('invalid url: ', resp2.url)

            soup2 = BeautifulSoup(resp2.text, 'html.parser')
            main_content2 = soup2.find(id = 'main-content')
            # metaline includes of author, title and datetime
            metas = main_content2.select('div.article-metaline')

            author = ''
            title = ''
            date = ''

            if metas:
                author = metas[0].select('span.article-meta-value')[0].string if metas[0].select('span.article-meta-value')[0] else author
                title = metas[1].select('span.article-meta-value')[0].string if metas[1].select('span.article-meta-value')[0] else title
                date = metas[2].select('span.article-meta-value')[0].string if metas[2].select('span.article-meta-value')[0] else date
                
                # 去除ptt中文暱稱
                author = re.sub('\s\(.*?\).', '', author)
                # deal with date format
                date = datetime.strptime(date, '%a %b  %d %H:%M:%S %Y')

                # remove meta nodes
                for m in metas:
                    m.clear()
                for mr in main_content2.select('div.article-metaline-right'):
                    mr.clear()
            
            # remove and keep push nodes
            for p in main_content2.find_all('div', class_='push'):
                p.clear()

            # 移除 '※ 發信站:' (starts with u'\u203b'), '◆ From:' (starts with u'\u25c6'), 空行及多餘空白
            # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
            filtered = [ v for v in main_content2.stripped_strings if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--'] ]
            expr = re.compile(u(r'[^\u4e00-\u9fa5\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\s\w:/-_.?~%()]'))
            for i in range(len(filtered)):
                filtered[i] = re.sub(expr, '', filtered[i])

            filtered = [_f for _f in filtered if _f]  # remove empty strings
            filtered = [x for x in filtered if article_id not in x]  # remove last line containing the url of the article
            content = ' '.join(filtered)
            content = re.sub(r'(\s)+', ' ', content)

            data = {
                'url': link,
                'title': title,
                'author': author,
                'date': date,
                'content': content
            }
            all_articles.append(data)
        except:
            print('This article is deleted: ', article_id)
        
    time.sleep(0.05)

# store data to pandas and write to csv
all_articles = pd.DataFrame(all_articles)
all_articles.head()
all_articles.to_csv('hate_ptt.csv', index=False)

In [87]:
all_articles = pd.DataFrame(all_articles)
# all_articles.to_csv('hate_ptt.csv', index=False)
all_articles
all_articles.to_csv('hate_ptt.csv', index=False)

,author,content,date,title,url
0,Kaohsiung85 (東帝士85超高大樓),哇勒，此人混蛋至極，叫他野蠻室友太客氣了， 靠，機掰的要死，沒有做到他的標準， 就開始罵髒話...,2003-04-06 20:45:59,恨！我的野蠻室友,https://www.ptt.cc/bbs/Hate/M.1049633636.A.41C...
1,YICHANCHUNG (sodisga),根本就是把庾澄慶踢掉的超級星期天嘛 感覺很像是幕後有很多角力鬥爭似的 雖然哈林主持功力不佳....,2003-04-06 21:35:45,什麼快樂星期天阿,https://www.ptt.cc/bbs/Hate/M.1049636067.A.255...
2,Evance (嘆),只能夠被動的拒絕面對 不能讓他遠離我的世界 我只好選擇把眼睛閉上 把耳朵關上 把心鎖上 無奈...,2003-04-06 21:51:59,我好懦弱,https://www.ptt.cc/bbs/Hate/M.1049636988.A.042...
3,lightthefire (我以為我的以為),幹你媽的樓下死小孩 你亂按什麼電玲你不隻到這樣很雞掰嗎 幹幹幹 幹林鄒罵吼哩驚到 害林背隔壁...,2003-04-06 22:18:11,幹你娘基八亂按什麼電玲啊幹,https://www.ptt.cc/bbs/Hate/M.1049638506.A.993...
4,geju (逝水),常常在疑惑，到底什麼是對?什麼又是錯呢? 最近我和室友處得不太好嗯或許該說是非常差。導因於上...,2003-05-18 10:15:18,[問題] 來自朋友的傷害果然很痛,https://www.ptt.cc/bbs/Hate/M.1053224089.A.71D...
5,alstonfju ( Je t'aime),幹你媽的..說那什麼白目話.. 我們都沒指控你們這群色胚來台灣買春玩女人.. 現在居然還用這...,2003-05-18 15:29:52,他媽的東洋倭寇死色鬼,https://www.ptt.cc/bbs/Hate/M.1053243321.A.2C4...
6,lyremoon (天地無涯),: 再看看之前版主 瑞被卡司他是怎麼作的 : 他一遇到灌水文，馬上就刪，結果都可以順利過關，...,2003-05-21 01:44:38,Re: 彼得，我對妳相當失望。(一個被妳禁水桶的人的꘠…,https://www.ptt.cc/bbs/Hate/M.1053450362.A.7CF...
7,bullock (Ghost Ring--咩咩),彼得，恨版在PTT也算是大版之一，管理一各大版又是公眾版本來就是很難的一件事情。 當個版主更...,2003-05-21 01:58:40,來自友板的建議。,https://www.ptt.cc/bbs/Hate/M.1053452148.A.7EE...
8,VoulezVous (Carrie),昨天跟同學看完電影之後， 跑到追分補習班去應徵一個好久以前就填過資料的電訪工作， 處理完我就...,2003-08-12 01:04:47,[心得] 昨天的公車302,https://www.ptt.cc/bbs/Hate/M.1060619545.A.AC6...
9,teimasako (貧窮富家女),我朋友真的很倒楣 去年大考前 遇到個交往3年的女友大背叛 每天哀聲哀調的跟我抱怨 大半夜還得...,2003-08-12 22:44:49,幫朋友po的--真是太衰了,https://www.ptt.cc/bbs/Hate/M.1060697956.A.10F...


## 以下為 testing code

In [33]:
#### TEST CODE #####
resp = requests.get(url = 'https://www.ptt.cc/bbs/Hate/M.1581756353.A.DC1.html', cookies={'over18':'1'})
# status_code=200 is success, so if not success will return error e.g. 404 not found
if resp.status_code != 200:
    print('invalid url: ', resp.url)

soup = BeautifulSoup(resp.text, 'html.parser')
main_content = soup.find(id = 'main-content')
# metaline includes of author, title and datetime
metas = main_content.select('div.article-metaline')
author = ''
title = ''
date = ''
if metas:
    author = metas[0].select('span.article-meta-value')[0].string if metas[0].select('span.article-meta-value')[0] else author
    title = metas[1].select('span.article-meta-value')[0].string if metas[1].select('span.article-meta-value')[0] else title
    date = metas[2].select('span.article-meta-value')[0].string if metas[2].select('span.article-meta-value')[0] else date#

    # remove meta nodes
#     for meta in metas:
#         meta.extract()
#     for meta in main_content.select('div.article-metaline-right'):
#         meta.extract()

In [34]:
#### TEST CODE #####

from six import u

article_id = 'M.1581756353.A.DC1'

# remove and keep push nodes
# pushes = main_content.find_all('div', class_='push')
# for push in pushes:
#     push.extract()
            
filtered = [ v for v in main_content.stripped_strings if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--'] ]
expr = re.compile(u(r'[^\u4e00-\u9fa5\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\s\w:/-_.?~%()]'))
for i in range(len(filtered)):
    filtered[i] = re.sub(expr, '', filtered[i])

filtered = [_f for _f in filtered if _f]  # remove empty strings
print(filtered)
filtered = [x for x in filtered if article_id not in x]  # remove last line containing the url of the article
content = ' '.join(filtered)
print(content)
content = re.sub(r'(\s)+', ' ', content)
print(content)

['作者', 'fengyu118 (煩魚)', '看板', 'Hate', '標題', '[黑特] 夢中夢', '時間', 'Sat Feb 15 16:45:51 2020', '小弟剛剛做了一個可怕的噩夢\n\n夢中小弟被人設計陷害\n\n堅持自己是真的遇到這種爛事\n\n可是沒有人相信我\n\n小弟開始懷疑自己是不是做夢夢到\n但我堅持不是假的\n\n結果醒來以後發現 QQ\n\n', 'https://www.ptt.cc/bbs/Hate/M.1581756353.A.DC1.html', '噓', 'angelgirl13', ':', 'http://i.imgur.com/N30aH6o.jpg', '02/15 16:46', '噓', 'celtics1997', ': 我現在好難過可以打打我嗎', '02/15 16:46', 'fengyu118', ': 13 很氣喔', '02/15 16:46', 'fengyu118', ': 打到你哭著叫媽', '02/15 16:47', '噓', 'lin91223', ': 煩魚我也要    ', '02/15 16:47', 'fengyu118', ': 不要 我要直接把你的毯毯搶走', '02/15 16:47', '推', 'celtics1997', ': 可以看了YAYAYA', '02/15 16:49', '噓', 'lin91223', ': 不行 我要裹著毯毯被打    ', '02/15 16:49', 'lpbrother', ': 我都夢到樓下', '02/15 16:51', 'celtics1997', ': 大家快去看Netflix YAYAYA', '02/15 16:51', '噓', 'celtics1997', ': Outlander上了YAYAYA 馬眼YA透ㄌ...', '02/15 16:55', 'celtics1997', ': 好爽喔喔喔喔喔喔喔', '02/15 16:55', 'celtics1997', ': 喔喔喔喔喔喔喔欸欸', '02/15 16:55', 'celtics1997', ': AAAAAAA', '02/15 16:55']
作者 fengyu118 (煩魚) 看板 Hate 標題 [黑特] 夢中夢 時間 Sa